In [ ]:
#default_exp data.load

In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.notebook.showdoc import show_doc

In [ ]:
#export
from torch.utils.data.dataloader import _MultiProcessingDataLoaderIter,_SingleProcessDataLoaderIter,_DatasetKind
_loaders = (_MultiProcessingDataLoaderIter,_SingleProcessDataLoaderIter)

In [ ]:
bs = 4
letters = list(string.ascii_lowercase)

## DataLoader

In [ ]:
#export
def _wif(worker_id):
    info = get_worker_info()
    ds = info.dataset.d
    ds.nw,ds.offs = info.num_workers,info.id
    set_seed(info.seed)
    ds.wif()

class _FakeLoader(GetAttr):
    _auto_collation,collate_fn,drop_last,dataset_kind,_index_sampler = False,noops,False,_DatasetKind.Iterable,Inf.count
    def __init__(self, d, pin_memory, num_workers, timeout):
        self.dataset,self.default,self.worker_init_fn = self,d,_wif
        store_attr(self, 'd,pin_memory,num_workers,timeout')
    def __iter__(self): return iter(self.d._iter())

In [ ]:
#export
_collate_types = (ndarray, Tensor, typing.Mapping, str)

In [ ]:
#export
def fa_collate(t):
    b = t[0]
    return (default_collate(t) if isinstance(b, _collate_types)
            else type(t[0])([fa_collate(s) for s in zip(*t)]) if isinstance(b, Sequence)
            else default_collate(t))

In [ ]:
#e.g. x is int, y is tuple
t = [(1,(2,3)),(1,(2,3))]
test_eq(fa_collate(t), default_collate(t))
test_eq(L(fa_collate(t)).mapped(type), [Tensor,tuple])

t = [(1,(2,(3,4))),(1,(2,(3,4)))]
test_eq(fa_collate(t), default_collate(t))
test_eq(L(fa_collate(t)).mapped(type), [Tensor,tuple])
test_eq(L(fa_collate(t)[1]).mapped(type), [Tensor,tuple])

In [ ]:
#export
def fa_convert(t):
    return (default_collate(t) if isinstance(t, _collate_types)
            else type(t)([fa_convert(s) for s in t]) if isinstance(t, Sequence)
            else default_convert(t))

In [ ]:
t0 = array([1,2])
t = [t0,(t0,t0)]

test_eq(fa_convert(t), default_convert(t))
test_eq(L(fa_convert(t)).mapped(type), [Tensor,tuple])

In [ ]:
#export
@methods_kwargs
class DataLoader():
    reset=item_tfm=batch_tfm=wif = noops
    _methods = 'collate_fn batches reset wif sampler item batch_tfm item_tfm'.split()
    def __init__(self, dataset=None, bs=None, drop_last=False, shuffle=False, indexed=None,
                 num_workers=0, pin_memory=False, timeout=0, retain=True, **kwargs):
        if indexed is None: indexed = dataset is not None and hasattr(dataset,'__getitem__')
        store_attr(self, 'dataset,bs,drop_last,shuffle,indexed,retain')
        self.fake_l = _FakeLoader(self, pin_memory, num_workers, timeout)
        self.lock,self.rng,self.nw,self.offs = Lock(),random.Random(),1,0
        try: self.n = len(self.dataset)
        except TypeError: self.n = None
        assert not kwargs and not (bs is None and drop_last)

    def __iter__(self): return _loaders[self.fake_l.num_workers==0](self.fake_l)
    def _iter(self):
        self.it = iter(self.dataset) if self.dataset else None
        self.reset()
        return self.batches()
    
    def __len__(self):
        if self.n is None: raise TypeError
        if self.bs is None: return self.n
        return self.n//self.bs + (0 if self.drop_last or self.n%self.bs==0 else 1)
    
    def batches(self):
        res = maps(self.item, self.item_tfm, self.sampler())
        if self.bs is None: return res
        else: return (self.batch_tfm(self.collate_fn(b))
                      for b in chunked(res, self.bs, self.drop_last))

    def shuffle_fn(self, idxs): return self.rng.sample(idxs, len(idxs))
    def sampler(self):
        idxs = Inf.count if self.indexed else Inf.nones
        if self.n is not None:
            idxs = list(itertools.islice(idxs, self.n))
            idxs = self.shuffle_fn(idxs) if self.shuffle else idxs
        return (b for i,b in enumerate(idxs) if i//(self.bs or 1)%self.nw==self.offs)

    def item(self, s): return next(self.it) if s is None else self.dataset[s]
    def collate_fn(self, b):
        res = (fa_collate,fa_convert)[self.bs is None](b)
        if self.retain and is_iter(b[0]): res = retain_types(res, b[0])
        return res

In [ ]:
class RandDL(DataLoader):
    def item(self, s):
        r = random.random()
        return r if r<0.95 else stop()

L(RandDL())

(#20) [0.5255343115652369,0.4326934165257216,0.06695079104228518,0.9463814453003001,0.5581177893825361,0.8351652321132943,0.7356221127488409,0.5498760817490563,0.07469770867842429,0.6364322506063604...]

Override `batches` to return some specific finite iterable.

In [ ]:
class LettersDL(DataLoader):
    def batches(self): return (string.ascii_lowercase[i:i+4] for i in range(0,26,4))

test_eq(L(LettersDL()), 'abcd,efgh,ijkl,mnop,qrst,uvwx,yz'.split(','))

Use `self.sampler()` to get indexes of samples, if needed. If you didn't pass a `dataset`  then `idxs` is just a list of `None`s of appropriate length.

In [ ]:
class RandDL(DataLoader):
    def batches(self): return gen(lambda o:random.random(), self.sampler(), lt(0.95))

L(RandDL())

(#2) [0.9208819226740764,0.8617893234228746]

Pass a method to `__init__` instead of overriding if you prefer.

In [ ]:
def _batches(self): return gen(lambda o:random.random(), self.sampler(), lt(0.95))
L(DataLoader(batches=_batches))

(#51) [0.31569666822407105,0.7288007970901603,0.03029330279056386,0.5106360984153411,0.11664760697588339,0.8533366887356388,0.8144846978500193,0.48776393579072674,0.8485945439388658,0.6964727698479298...]

Override `item` and use the default infinite sampler to get a stream of unknown length (`stop()` when you want to stop the stream).

In [ ]:
class RandDL(DataLoader):
    def item(self, s):
        r = random.random()
        return r if r<0.95 else stop()

L(RandDL())

(#58) [0.050259100488361574,0.8257104607987692,0.6736799488072958,0.42660923983121446,0.8440959460795934,0.6325967519266268,0.29884691835507826,0.6196327749318266,0.23361553109596944,0.02289621490279725...]

In [ ]:
L(RandDL(bs=4, drop_last=True)).mapped(len)

(#5) [4,4,4,4,4]

In [ ]:
L(RandDL(bs=4, num_workers=4, drop_last=True)).mapped(len)

(#7) [4,4,4,4,4,4,4]

If you don't set `bs`, then `dataset` is assumed to provide an iterator or a `__getitem__` that returns a batch.

In [ ]:
ds1 = DataLoader(letters)
test_eq(L(ds1), letters)
test_eq(len(ds1), 26)

test_shuffled(L(DataLoader(letters, shuffle=True)), letters)

ds1 = DataLoader(letters, indexed=False)
test_eq(L(ds1), letters)
test_eq(len(ds1), 26)

t2 = L(tensor([0,1,2]),tensor([3,4,5]))
ds2 = DataLoader(t2)
test_eq_type(L(ds2), t2)

t3 = L(array([0,1,2]),array([3,4,5]))
ds3 = DataLoader(t3)
test_eq_type(L(ds3), t2)

ds4 = DataLoader(t3, collate_fn=noops)
test_eq_type(L(ds4), t3)

If you do set `bs`, then `dataset` is assumed to provide an iterator or a `__getitem__` that returns a single item of a batch.

In [ ]:
def twoepochs(d): return ' '.join(''.join(o) for _ in range(2) for o in d)

In [ ]:
ds1 = DataLoader(letters, bs=4,drop_last=True, num_workers=0)
test_eq(twoepochs(ds1), 'abcd efgh ijkl mnop qrst uvwx abcd efgh ijkl mnop qrst uvwx')

ds1 = DataLoader(letters,4,num_workers=2)
test_eq(twoepochs(ds1), 'abcd efgh ijkl mnop qrst uvwx yz abcd efgh ijkl mnop qrst uvwx yz')

ds1 = DataLoader(range(12), bs=4, num_workers=3)
test_eq_type(L(ds1), L(tensor([0,1,2,3]),tensor([4,5,6,7]),tensor([8,9,10,11])))

ds1 = DataLoader([str(i) for i in range(11)], bs=4, num_workers=5)
test_eq_type(L(ds1), L(['0','1','2','3'],['4','5','6','7'],['8','9','10']))

it = iter(DataLoader(map(noop,range(20)), bs=4, num_workers=1))
test_eq_type([next(it) for _ in range(3)], [tensor([0,1,2,3]),tensor([4,5,6,7]),tensor([8,9,10,11])])

In [ ]:
class SleepyDL(list):
    def __getitem__(self,i):
        time.sleep(random.random()/50)
        return super().__getitem__(i)

t = SleepyDL(letters)

%time test_eq(DataLoader(t, num_workers=0), letters)
%time test_eq(DataLoader(t, num_workers=2), letters)
%time test_eq(DataLoader(t, num_workers=4), letters)
test_shuffled(L(DataLoader(t, shuffle=True, num_workers=4)), letters)

CPU times: user 3.2 ms, sys: 1.76 ms, total: 4.96 ms
Wall time: 253 ms
CPU times: user 14.6 ms, sys: 13.6 ms, total: 28.2 ms
Wall time: 171 ms
CPU times: user 14.4 ms, sys: 19.5 ms, total: 33.9 ms
Wall time: 102 ms


In [ ]:
class SleepyQueue():
    "Simulate a queue with varying latency"
    def __init__(self, q): self.q=q
    def __iter__(self):
        while True:
            time.sleep(random.random()/100)
            try: yield self.q.get_nowait()
            except queues.Empty: return

q = Queue()
for o in range(30): q.put(o)
it = SleepyQueue(q)

%time test_shuffled(L(DataLoader(it, num_workers=4)), range(30))

CPU times: user 13.1 ms, sys: 20.2 ms, total: 33.3 ms
Wall time: 94 ms


## Export -

In [ ]:
#hide
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core.ipynb.
Converted 01a_dataloader.ipynb.
Converted 01a_script.ipynb.
Converted 02_transforms.ipynb.
Converted 03_pipeline.ipynb.
Converted 04_data_external.ipynb.
Converted 05_data_core.ipynb.
Converted 06_data_source.ipynb.
Converted 07_vision_core.ipynb.
Converted 08_pets_tutorial.ipynb.
Converted 09_vision_augment.ipynb.
Converted 09a_rect_augment.ipynb.
Converted 10_data_block.ipynb.
Converted 11_layers.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 14_callback_schedule.ipynb.
Converted 15_callback_hook.ipynb.
Converted 16_callback_progress.ipynb.
Converted 17_callback_tracker.ipynb.
Converted 18_callback_fp16.ipynb.
Converted 19_callback_mixup.ipynb.
Converted 20_metrics.ipynb.
Converted 21_tutorial_imagenette.ipynb.
Converted 30_text_core.ipynb.
Converted 31_text_data.ipynb.
Converted 32_text_models_awdlstm.ipynb.
Converted 33_test_models_core.ipynb.
Converted 34_callback_rnn.ipynb.
Converted 35_tutorial_wikitex